### Import libraries

In [47]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import csv
from bs4 import BeautifulSoup
from textblob import TextBlob

### Define useful functions

In [65]:
def extract_review_body(review_query):
#     if review_query != None:
#         review_text_tmp = review_query.get_text().replace('\n','.')
#         if len(review_text_tmp)<15:
#             return None
#         else:
#             if TextBlob(review_text_tmp[:15]).detect_language()=='en':
#                 return review_text_tmp
#             else:
#                 return None
#     else:
#         return None
    if review_query != None:
        return review_query.get_text().replace('\n','.')
    else:
        return None

def extract_review_staydate(review_query):
    if review_query != None:
        return review_query.get_text().strip('\n').strip('Check-in ')
    else:
        return None
    
def de_emojify(inputString):
    if inputString!=None:
        return inputString.encode('ascii', 'ignore').decode('ascii')
    else:
        return inputString

In [41]:
def get_reviews_input_hotel(input_url):
    
    input_hotel_reviews_list_csvname = './datasets/input/input_hotel_reviews_list.csv'

    with open(input_hotel_reviews_list_csvname, 'w', newline='', encoding="UTF-8") as f:
        writer = csv.writer(f)
        writer.writerow(['hotel_name','hotel_category_scores',
                     'review_score_badge','review_item_header_content','review_info_tag','review_staydate',
                     'review_body'])
    
        next_page=True
        ip=1
        hotel_reviews_list = []
        while next_page:
            url= input_url + '-tr-p' + str(ip) + '/?applyEmbargo=true&amp;reviewTab=brand-reviews'
            response = requests.get(url,headers=headers)
            hotel_reviews = BeautifulSoup(response.text, 'html.parser')
            
            if ip==1:
                hotel_category_scores_list_html = hotel_reviews.find_all(class_='category')
                hotel_category_scores_list = []
                for item in hotel_category_scores_list_html:
                    hotel_category_scores_list.append([item.find(class_='category-name').get_text(),item.find(class_='category-score').get_text()])
            
            hotel_reviews_page = hotel_reviews.find_all(class_=['review-card'])
            hotel_reviews_list.extend(hotel_reviews_page)
            ip+=1
            if hotel_reviews_page==[]:
                next_page=False
                
        for hotel_reviews_item in hotel_reviews_list:
       
            review_score_badge = hotel_reviews_item.find(class_ = 'rating-score').get_text()
            review_item_header_content = hotel_reviews_item.find(class_ = 'rating-badge').get_text().strip('\n')
            review_staydate = extract_review_staydate(hotel_reviews_item.find(class_ = 'date'))            
            review_body = de_emojify(extract_review_body(hotel_reviews_item.find(class_ = 'expandable-content description')))            
            review_info_tag = hotel_reviews_item.find(class_='trip-type-nights').get_text()
            
            writer.writerow([input_url.split('hotels.com/')[1],hotel_category_scores_list,review_score_badge,
                             review_item_header_content,review_info_tag,review_staydate,review_body]) 

In [42]:
def get_info_input_hotel(input_url):
    input_hotel_info_csvname = './datasets/input/input_hotel_info.csv'

    with open(input_hotel_info_csvname, 'w', newline='', encoding="UTF-8") as f:
        writer = csv.writer(f)
        writer.writerow(['hotel_name','hotel_name_full','hotel_stars','hotel_address',
                     'hotel_overall_rating','hotel_room_name','hotel_room_price'])
    
        url= input_url + '/?q-check-out=2020-09-17&q-check-in=2020-09-16'
        response = requests.get(url,headers=headers)
        hotel_info = BeautifulSoup(response.text, 'html.parser')
    
        hotel_name_full = hotel_info.find(class_='vcard').get_text()
        hotel_stars = hotel_info.find(class_='star-rating-text widget-star-rating-overlay widget-tooltip widget-tooltip-responsive widget-tooltip-ignore-touch').get_text()
        hotel_address = hotel_info.find(class_='postal-addr').get_text()
        hotel_overall_rating = hotel_info.find(class_='rating good').get_text()
        hotel_room_name = hotel_info.find(class_='room-name').get_text()
        hotel_room_price = hotel_info.find(class_='current-price').get_text()
    
        writer.writerow([input_url.split('hotels.com/')[1],hotel_name_full,hotel_stars,
                         hotel_address,hotel_overall_rating,hotel_room_name,hotel_room_price])

### Specify user agent

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

### User input

In [43]:
input_url = 'https://www.hotels.com/ho127108'

### Get reviews of input hotel

In [66]:
get_reviews_input_hotel(input_url)

### Get info of input hotel

In [45]:
get_info_input_hotel(input_url)